In [492]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor  # or RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer

In [493]:
# lOADING
train_test = pd.read_csv("../processing_2/data_all.csv", parse_dates=['Date'])

In [494]:
len(train_test)

3943

In [495]:
# Tonnage to float
train_test['Tonnage'] = train_test['Tonnage'].str.replace(',', '.').astype(float)

# Split the date
train_test['Year'] = train_test['Date'].dt.year
train_test['Month'] = train_test['Date'].dt.month
train_test['Day'] = train_test['Date'].dt.day


duplicates = train_test.duplicated(['CodeTracabilite', 'Num Parcelle', 'Campagne', 'Region', 'Site', 'Famille', 'Variete', 'Recolte'], keep=False)

# Then, apply the condition
condition = (train_test['Sup Debut Camp'] + train_test['Sup Plantee'] != train_test['Sup Arrachee'] + train_test['Sup Fin Camp'])

# Combine both conditions
to_drop = duplicates & condition

# Drop the rows that satisfy both conditions
train_test = train_test[~to_drop]

In [496]:
for month_index in range(12):
    train_test[f'irrigation_cumulee_{month_index + 1}'] = train_test[f'irrigation_cumulee_{month_index + 1}'].replace(0, np.nan)

In [497]:
# for month_index in range(12):
#     # print(f'Len irrigation {month_index + 1}:', len(train_test[train_test[f'irrigation_cumulee_{month_index + 1}'] == 0]))
    # print(f'Len irrigation {month_index + 1}:', len(train_test[train_test[f'irrigation_cumulee_{month_index + 1}'].isna()]))

In [498]:
# irrigation_columns = [f'irrigation_cumulee_{i}' for i in range(1, 13)]

# # Calculate the median for these columns in each row
# row_medians = train_test[irrigation_columns].median(axis=1)

# # Replace NaN values in each column with the row median
# for col in irrigation_columns:
#     train_test[col] = train_test[col].fillna(row_medians)

In [499]:
cat_cols = ['Variete', 'Month', 'Year', 'Region', 'Site', 'Porte Greffe']

num_cols = ['Sup Debut Camp', 'Tree Age', 'Days to Arrachage', 'Recolte']

for index in range(12):
    num_cols.append(f'irrigation_cumulee_{index + 1}')
    num_cols.append(f'N_cumulee_{index + 1}')
    num_cols.append(f'P_cumulee_{index + 1}')
    num_cols.append(f'K_cumulee_{index + 1}')

target_col = ['Tonnage']

all_feature_cols = num_cols + cat_cols

In [500]:
print(f'''
      ▶️ Len nan sup debut camp      : {len(train_test[train_test['Sup Debut Camp'].isna()])}
      ▶️ Len nan Sup Plantee         : {len(train_test[train_test['Sup Plantee'].isna()])}
      ▶️ Len nan Sup Arrachee        : {len(train_test[train_test['Sup Arrachee'].isna()])}
      ▶️ Len nan Sup Fin Camp        : {len(train_test[train_test['Sup Fin Camp'].isna()])}
      ▶️ Len nan Days to Arrachage   : {len(train_test[train_test['Days to Arrachage'].isna()])}
      ▶️ Len nan Tree Age            : {len(train_test[train_test['Tree Age'].isna()])}
''')


      ▶️ Len nan sup debut camp      : 365
      ▶️ Len nan Sup Plantee         : 365
      ▶️ Len nan Sup Arrachee        : 365
      ▶️ Len nan Sup Fin Camp        : 365
      ▶️ Len nan Days to Arrachage   : 2595
      ▶️ Len nan Tree Age            : 467



for day to arachage, i'll apply the median 
for others i need to see the distribution of these nan values, then decide


Pour sup debut camp:365
- these are the regions of nan values: ['HAOUZ', 'SOUSS', 'TADLA', 'ORIENTAL']
    - HAOUZ: 300
        - these NaN are in these 3 Site: ['Marrakech', 'Ait Ourir', 'Ouled Moussa']
            - Marrakech: 62
            - Ait Ourir: 1
            - Ouled Moussa: 237  (whereas 225 others are not NaN)
    - SOUSS: 17
    - Tadla: 47
    - ORENTAL: 1

    ▶️ I'll use KNN 

conclusion: les nan values de Sup Fin Camp, Sup Arrachee, Sup Plantee, Sup debut comp se coincide

    ▶️ I'll use for all of them KNN

- Tadla: 47 sameas sup debut camp
- Oriental: 1  sameas sup debut camp
- Gharb: 1
- SOUSS: 29   Not too uch compared with the none nan vals
- HAOUZ: 390
    - Ouled Moussa: 237 (same thing)
    - Lalla Takerkoust: 4
    - Douar Ahl quadi: 41
    - Marrakech: 107
    - Ait Ourir: 1

▶️ KNN

## Pipeline

Preprocessing

In [501]:
cols_to_impute_knn = ['Sup Debut Camp']
cols_to_impute_median = ['Days to Arrachage', 'Tree Age']

for index in range(12):
    cols_to_impute_median.append(f'irrigation_cumulee_{index + 1}')

In [502]:
# Define the transformer
cat_transformer = OneHotEncoder(sparse=False, handle_unknown='ignore')

median_imputer = SimpleImputer(strategy='median')

# KNNImputer for numerical features
knn_imputer = KNNImputer(n_neighbors=5)

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_cols),
        ('impute_median', median_imputer, cols_to_impute_median),
        ('impute_knn', knn_imputer, cols_to_impute_knn),
    ],
    remainder='passthrough'
)


train = train_test[train_test['is_train']]
test = train_test[~train_test['is_train']]

X = train[all_feature_cols]
y = train[target_col]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_test = test[all_feature_cols]

In [503]:
train_test.isna().sum()

Campagne                0
Region                  0
Site                    0
Famille                 0
Variete                 0
                     ... 
Tree Age              467
Days to Arrachage    2595
Year                    0
Month                   0
Day                     0
Length: 72, dtype: int64

In [483]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

pipeline = make_pipeline(preprocessor, model)

pipeline.fit(X_train, y_train)

c:\Users\DataScience\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor


# models = [RandomForestRegressor(), XGBRegressor(), LinearRegression(), LGBMRegressor()]

In [ ]:
# from sklearn.metrics import r2_score

# # Create the complete pipeline
# for model in models:
#     pipeline = make_pipeline(preprocessor, model)
#     pipeline.fit(X_train, y_train)
#     y_train_pred = pipeline.predict(X_train)

#     evaluation_metrics(y_train, y_train_pred)

In [ ]:
# for model in models:
#     pipeline = make_pipeline(preprocessor, model)
#     pipeline.fit(X_train, y_train)
#     y_val_pred = pipeline.predict(X_val)

#     print(f'✅ model: {model}')
#     evaluation_metrics(y_val, y_val_pred)

In [ ]:
# from lightgbm import LGBMRegressor
# model = LGBMRegressor()

In [ ]:
y_train_pred = pipeline.predict(X_train)

In [ ]:
from sklearn.metrics import r2_score

def evaluation_metrics(y, y_pred):
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    r_squared = r2_score(y, y_pred)
    std = train_test[train_test['is_train']]['Tonnage'].std()


    r_squared
    print(f'''     
        ▶️ MSE  : {mse}
        ▶️ RMSE : {rmse} vs STD  : {std}   
        ▶️ R²   : {r_squared}
        ''')

In [ ]:
evaluation_metrics(y_train, y_train_pred)

     
        ▶️ MSE  : 224367547.12843108
        ▶️ RMSE : 14978.903402066224 vs STD  : 38941.21577045369   
        ▶️ R²   : 0.8182186756849181
        


In [ ]:
y_val_pred = pipeline.predict(X_val)

In [ ]:
evaluation_metrics(y_val, y_val_pred)

     
        ▶️ MSE  : 1987219056.392059
        ▶️ RMSE : 44578.235231916246 vs STD  : 38941.21577045369   
        ▶️ R²   : 0.24637172608656188
        


In [464]:
# # Access the linear regression model
# model = pipeline.named_steps['linearregression']

# # Extract the ColumnTransformer
# preprocessor = pipeline.named_steps['columntransformer']

# # Initialize an empty list to hold feature names
# transformed_feature_names = []

# # Iterate through each transformer in the ColumnTransformer
# for transformer_name, transformer, columns in preprocessor.transformers_:
#     if transformer_name != 'remainder':
#         if hasattr(transformer, 'get_feature_names_out'):
#             # If the transformer can generate feature names, use it
#             transformed_feature_names.extend(transformer.get_feature_names_out(columns))
#         else:
#             # Otherwise, use the original column names
#             transformed_feature_names.extend(columns)
#     else:
#         # For the 'remainder' transformer, add the remaining column names
#         transformed_feature_names.extend(columns)

# # If 'remainder' is 'passthrough', include remaining column names
# if preprocessor.remainder == 'passthrough':
#     remaining_cols = [col for col in X_train.columns if col not in cat_cols and col not in cols_to_impute]
#     transformed_feature_names.extend(remaining_cols)

# # Assuming model.coef_ contains the coefficients
# for feature_name, coef in zip(transformed_feature_names, model.coef_):
#     print(f'{feature_name}: {coef}')

In [469]:
y_pred = pipeline.predict(X_test)

In [470]:
y_pred.shape

(781,)

In [471]:
y_pred = np.array(y_pred)  # Convert to numpy array if not already
assert len(y_pred) == 781, "y_pred must have exactly 781 rows"

# Create a DataFrame. Adjust column names as per the competition's requirement.
# Usually, you will have an ID column and a prediction column.
submission_df = pd.DataFrame({
    'Id': range(1, 782),  # Example: creating a sequence of IDs from 1 to 781
    'Tonnage': y_pred
})

# Export to CSV
csv_file = "submission.csv"
submission_df.to_csv(csv_file, index=False)